This code is to do the one time check described in [ticket 1917]( Run a one-time check on count of federal Indian reservations)

> We should do a one-time check on the BIA data we import, such that after import that have (for now in our staging environment) 326 federal Indian reservations on our map. The number 326 from https://www.bia.gov/faqs/what-federal-indian-reservation.
>
> If that one-time check to make sure we have all reservations fails, there's a list of federal Indian reservation names that we can use to track down which reservations are not shown on our map and troubleshoot.

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from data_pipeline.etl.sources.tribal.etl import TribalETL
from data_pipeline.etl.sources.geo_utils import get_tribal_geojson
import geopandas as gpd

In [3]:
%load_ext lab_black

In [4]:
etl = TribalETL()
etl.extract()

2022-09-23 11:56:25,551 [data_pipeline.etl.sources.tribal.etl] INFO     Downloading Tribal Data
2022-09-23 11:56:25,552 [data_pipeline.utils] INFO     Downloading https://justice40-data.s3.amazonaws.com/data-sources/BIA_National_LAR_json.zip
2022-09-23 11:56:26,068 [data_pipeline.utils] INFO     Extracting /home/matt/active/justice40-tool/data/data-pipeline/data_pipeline/data/tmp/downloaded-47ce415c-cc72-4e6f-9cbc-7ad833e08813.zip
2022-09-23 11:56:26,190 [data_pipeline.utils] INFO     Downloading https://justice40-data.s3.amazonaws.com/data-sources/Alaska_Native_Villages_json.zip
2022-09-23 11:56:26,290 [data_pipeline.utils] INFO     Extracting /home/matt/active/justice40-tool/data/data-pipeline/data_pipeline/data/tmp/downloaded-7c700e59-83cd-4752-889c-159e58c71154.zip


In [5]:
GEOJSON_BASE_PATH = etl.GEOJSON_BASE_PATH

In [6]:
bia_national_lar_geojson = (
    GEOJSON_BASE_PATH / "bia_national_lar" / "BIA_National_LAR.json"
)
bia_aian_supplemental_geojson = (
    GEOJSON_BASE_PATH / "bia_national_lar" / "BIA_AIAN_Supplemental.json"
)
bia_tsa_geojson_geojson = (
    GEOJSON_BASE_PATH / "bia_national_lar" / "BIA_TSA.json"
)
alaska_native_villages_geojson = (
    GEOJSON_BASE_PATH
    / "alaska_native_villages"
    / "AlaskaNativeVillages.gdb.geojson"
)

In [7]:
bia_national_lar_df = gpd.read_file(bia_national_lar_geojson)

In [8]:
len(
    sorted(
        list(
            bia_national_lar_df.LARName.str.replace(
                r"\(.*\) ", "", regex=True
            ).unique()
        )
    )
)

326

Looking at the main BIA LAR data file, and removing paranthecials (so that `'Acoma (Red Lake) LAR','Acoma LAR'` are counted as a single tribal entry), **we have 326 tribal areas**, which is the number we expected.